# Setup

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

llm_model = "gpt-3.5-turbo"

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# ConversationBufferMemory

This lesson is all about getting the LLM to be able to "remember" context over a longer time horizon in order to work on more complex applications or tasks.

In [3]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory_1 = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory_1,
    verbose=True
)

C:\Users\hjfen\AppData\Local\Temp\ipykernel_3756\1199577467.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.0, model=llm_model)
C:\Users\hjfen\AppData\Local\Temp\ipykernel_3756\1199577467.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_1 = ConversationBufferMemory()
C:\Users\hjfen\AppData\Local\Temp\ipykernel_3756\1199577467.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_cor

In [4]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [5]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [6]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. Is there anything else you would like to know or discuss?'

As you can see this mimics the way the main GenAI companies design the main public facing apps.

In [7]:
print(memory_1.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Andrew. Is there anything else you would like to know or discuss?


In [8]:
memory_1.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew. Is there anything else you would like to know or discuss?"}

We can actually create a memory from scratch and manually add context to it.

In [9]:
memory_2 = ConversationBufferMemory()
memory_2.save_context({"input": "Hi"}, 
                    {"output": "What's up"})
print(memory_2.buffer)

Human: Hi
AI: What's up


In [10]:
memory_2.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

We can do it again.

In [11]:
memory_2.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [12]:
memory_2.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

Large Language Models are stateless, meaning each transaction is independent. Chatbots appear to have memory by providing the full conversation as "context".

# ConversationBufferWindowMemory

In [13]:
from langchain.memory import ConversationBufferWindowMemory

This type of memory only keeps a certain number of exchanges in memory to reduce the amount of tokens sent back and forth to the LLM. This "window" size is controlled with the parameter k.

In [14]:
memory_3 = ConversationBufferWindowMemory(k=1)  

C:\Users\hjfen\AppData\Local\Temp\ipykernel_3756\2357733892.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_3 = ConversationBufferWindowMemory(k=1)


In [15]:
memory_3.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory_3.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [16]:
memory_3.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [17]:
llm_2 = ChatOpenAI(temperature=0.0, model=llm_model)
memory_4 = ConversationBufferWindowMemory(k=1)
conversation_2 = ConversationChain(
    llm=llm_2, 
    memory = memory_4,
    verbose=False
)

In [18]:
conversation_2.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [19]:
conversation_2.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

It forgets the name because that exchange is outside of the window.

In [20]:
conversation_2.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to ask?"

# ConversationTokenBufferMemory

This type of memory limits the number of tokens that are saved in memory. This is controlled by the max_token_limit parameter

In [21]:
from langchain.memory import ConversationTokenBufferMemory

In [22]:
llm_3 = ChatOpenAI(temperature=0.0, model=llm_model)

In [23]:
# Different providers have different ways of counting tokens so we need to provide the llm type as well
memory_5 = ConversationTokenBufferMemory(llm=llm, max_token_limit=50) 
memory_5.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory_5.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory_5.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

C:\Users\hjfen\AppData\Local\Temp\ipykernel_3756\437866189.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_5 = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)


In [24]:
memory_5.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

# ConversationSummaryMemory

Instead of limiting the memory directly, lets use an llm to write a summary of the conversation so far and then use that as the memory.

In [25]:
from langchain.memory import ConversationSummaryBufferMemory

In [26]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory_6 = ConversationSummaryBufferMemory(llm=llm_3, max_token_limit=100) # max_token_limit controls when the summary is triggered
memory_6.save_context({"input": "Hello"}, {"output": "What's up"})
memory_6.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory_6.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

C:\Users\hjfen\AppData\Local\Temp\ipykernel_3756\462298803.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_6 = ConversationSummaryBufferMemory(llm=llm_3, max_token_limit=100) # max_token_limit controls when the summary is triggered


In [27]:
memory_6.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.'}

In [28]:
conversation_3 = ConversationChain(
    llm=llm_3, 
    memory=memory_6,
    verbose=True
)

In [29]:
conversation_3.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.
Human: What would be a good demo to show?
AI:

> Finished chain.


"For the meeting with the product team, a good demo to show would be the latest prototype of the LangChain project, highlighting its key features such as real-time language translation, voice recognition, and text analysis capabilities. This demo will showcase the progress we've made and the potential impact of the project. Additionally, for the lunch meeting with the customer interested in AI, a demo showcasing our AI-powered customer service chatbot could be impressive. It can demonstrate how our technology can improve customer interactions and streamline support processes."

In [30]:
memory_6.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. For the meeting with the product team, a good demo to show would be the latest prototype of the LangChain project, highlighting its key features such as real-time language translation, voice recognition, and text analysis capabilities. This demo will showcase the progress we've made and the potential impact of the project. Additionally, for the lunch meeting with the customer interested in AI, a demo showcasing our AI-powered customer service chatbot could be impressive. It can demonstrate how our technology can improve customer interactions and streamline support processes."}

# Additional Memory Types

- Vector data memory: stores text in a vector database and retrieves the most relevant blocks of text
- Entity memories: using an LLM it remembers details about specific entities

Can also mix and match memory types and use them simultaneously.

Can also store conversations in a convention database